# Travelling salesman problem

Specifications of the problem : [https://en.wikipedia.org/wiki/Travelling_salesman_problem](https://en.wikipedia.org/wiki/Travelling_salesman_problem)

## Librairies

Here's all the libraries needed.

In [24]:
from itertools import combinations
import pandas as pd
import numpy as np
from icecream import ic
from geopy.distance import geodesic

## Initialization

We extract the cities from the file and create a matrix to store the distance between each city.

In [25]:
FILE = 'cities/vanuatu.csv'

CITIES = pd.read_csv(FILE, header=None, names=['name', 'lat', 'lon'])

DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))

for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km

## First greedy algorithm

At first, I just wanted a working solution in order to upgrade it step by step. 

So I define a fist greedy function that solves a tsp by going always the closest city.

In [26]:
def greedy_tsp(dist_matrix) -> list :
    """Function that return a list of indexes to solve a tsp on a given matrix of distances"""
    dist = dist_matrix.copy()
    city = 0
    tsp = list()

    while not np.all(dist == np.inf):
        tsp.append(city)
        dist[:, city] = np.inf
        closest = np.argmin(dist[city])
        city = closest
    
    tsp.append(tsp[0]) # Add the first city to close the hamiltonian cycle
    return tsp

Let's see what it does.

In [27]:
city_names = np.array([c['name'] for _, c in CITIES.iterrows()])

tsp = greedy_tsp(DIST_MATRIX)

tot_cost = 0
for c1, c2 in zip(tsp, tsp[1:]):
    print(f"{city_names[c1]} -> {city_names[c2]}")
    tot_cost += DIST_MATRIX[c1, c2]
print(f"Total cost : {tot_cost}")
    

Isangel -> Vila
Vila -> Lakatoro
Lakatoro -> Norsup
Norsup -> Luganville
Luganville -> Port Olry
Port Olry -> Longana
Longana -> Sola
Sola -> Isangel
Total cost : 1475.528091104531


As we can see, this is pretty fast, indeed the temporal complexity of the algorithm is pretty much in $O(N^2)$, where N is the number of cities.

But this is surely not an efficient algorithm since the cost of the final result is high.